First, load the data,in this notebook we are using the smokoy mountain data as the example. The modeling result will be fine tuned

In [1]:
import sys
sys.path.append('../../../Code')
import loadData 
import RunML
import RunML_continue
import FS
import metric

In [2]:
import pandas as pd
import numpy as np
import random

In [3]:
data,burn_label,un_label,duration_label,ASVs,df=loadData.loadSoilData("../data/count_table/merge_proportion_asv_smoky_moutain.csv")

In [4]:
data

array([[1.5786533e-02, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.6912335e-02, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 5.0700000e-05,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        8.2400000e-05, 8.2400000e-05],
       [1.5771320e-03, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00]])

In [5]:
print(data.shape)
print(burn_label.shape)

(59, 1768)
(59,)


In [6]:
# abundance matrix--> relative abundance matrix
data=FS.relative_abundance(data)
print(np.shape(data))

(59, 1768)


In [7]:
data

array([[0.01578653, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01691234, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [8]:
yList= np.column_stack((burn_label,un_label,duration_label))# y list is a 2D array, each column is a response outcome

print(yList[:5])
      
for i in range(yList.shape[1]):
    print(pd.Series(yList[:,i]).value_counts())

[['No' 'Natural' 'Annual']
 ['No' 'Natural' 'Perennial']
 ['No' 'Urban' 'Perennial']
 ['No' 'Urban' 'Annual']
 ['No' 'Urban' 'Perennial']]
Yes    42
No     17
Name: count, dtype: int64
Urban      33
Natural    26
Name: count, dtype: int64
Perennial    37
Annual       22
Name: count, dtype: int64


In [9]:
y_index=['burn_label', 'un_label', 'duration_label']

In [10]:
ASVs = np.array(ASVs)
ASVs

array(['ASV1', 'ASV2', 'ASV3', ..., 'ASV3215', 'ASV3216', 'ASV3217'],
      dtype='<U7')

For each y label, using Lasso with hyperparameter tuning to select the features: get the column numbers and the subset, then use SVM and RandomForest to model. 

### Fearure selection using Lasso, each result has 2 elements: array and column number

In [11]:
selectedlasso_burn = RunML_continue.LassoFS_CV(data,burn_label)

Best alpha: 0.1


In [12]:
selectedlasso_urban = RunML_continue.LassoFS_CV(data,un_label)

Best alpha: 0.1


In [13]:
selectedlasso_duration = RunML_continue.LassoFS_CV(data,duration_label)

Best alpha: 0.1


### 2. Kruskal-Wallis H-test on OTUs for each response variable.


In [14]:
# calculating H score for each OTU, the function will: 1. filter those below threshold (1%) to 0; 2. rank the data; calculate the H statistics
weights=FS.OTU_H_Score_fun(data,yList)

In [15]:
weights.shape

(3, 1768)

In [16]:
selectedOTU_index, eps=FS.indice_H_multisig(weights,yList)
print(eps)

86


In [17]:
# the subset based on the method.
X_FS = data[:,selectedOTU_index]

### 3. Model

In [18]:
cls = ['RF','SVM']
datatypes=['AllFeatures', 'SelectMicro', 'Lasso_finetune']

#### 3.1  Model for Burn

In [19]:
targetLabel=burn_label
result_burn = {}

In [ ]:
for clf in cls:
    result_burn[clf]={}
    result_burn[clf][AllFeatures] = RunML_continue.nested_loocv(data,yList[:,0],classifier=clf,smote=True)
    result_burn[clf][SelectMicro] = RunML_continue.nested_loocv(X_FS,yList[:,0],classifier=clf,smote=True)
    result_burn[clf][Lasso_finetune] = RunML_continue.nested_loocv(selectedlasso_burn[0],yList[:,0],classifier=clf,smote=True)

In [ ]:
print("Burn analysis result:")
for clf in cls:
    for dataset in datatypes:
        print(f"Mean Accuracy of {clf}_{dataset}:", result_burn[clf][dataset]["mean_accuracy"])
        print(f"Std Accuracy of {clf}_{dataset}:",np.std(result_burn[clf][dataset]["outer_accuracies"]))
        metric.plot_confusion_matrices(result_burn[clf][dataset]["y_true"], result_burn[clf][dataset]["y_pred"],f"{clf}_{dataset}",pos_y=1)


In [ ]:
result_burn_random = {}
iteration_random =30

for clf in cls:
    acc_list = []
    acc_std_list = []
    random.seed(1992)
    for _ in range(iteration_random):
        selected_indices = random.sample(range(np.shape(data)[1]), np.shape(X_FS)[1])
        new_df = data[:, selected_indices]
        result_random_run = RunML_continue.nested_loocv(new_df,yList[:,0],classifier=clf,smote=True)
        acc_eachrun  = result_random_run["mean_accuracy"]
        acc_std_eachrun = np.std(result_random_run["outer_accuracies"])
        acc_list.append(acc_eachrun)
        acc_std_list.append(acc_std_eachrun)
    result_burn_random[clf] = {"average_accuracy": sum(acc_list) / len(acc_list) ,
                              "average_std_of_accuracy": sum(acc_std_list) / len(acc_std_list)}


In [ ]:
print("Burn analysis of random selected features result:")
for clf in cls:
    print(f"Average Accuracy of {clf} in random selection:", result_burn_random[clf]["average_accuracy"])
    print(f"Average std of Accuracy of {clf} in random selection:",result_burn_random[clf]["average_std_of_accuracy"])
    

#### 3.2 Model of Urban

In [ ]:
targetLabel=un_label
result_urban = {}

In [ ]:
for clf in cls:
    result_urban[clf]={}
    result_urban[clf][AllFeatures] = RunML_continue.nested_loocv(data,yList[:,1],classifier=clf,smote=True)
    result_urban[clf][SelectMicro] = RunML_continue.nested_loocv(X_FS,yList[:,1],classifier=clf,smote=True)
    result_urban[clf][Lasso_finetune] = RunML_continue.nested_loocv(selectedlasso_urban[0],yList[:,1],classifier=clf,smote=True)

In [ ]:
print("Urban analysis result:")
for clf in cls:
    for dataset in datatypes:
        print(f"Mean Accuracy of {clf}_{dataset}:", result_urban[clf][dataset]["mean_accuracy"])
        print(f"Std Accuracy of {clf}_{dataset}:",np.std(result_urban[clf][dataset]["outer_accuracies"]))
        metric.plot_confusion_matrices(result_urban[clf][dataset]["y_true"], result_urban[clf][dataset]["y_pred"],f"{clf}_{dataset}",pos_y=1)


#### 3.3 Model for Annual

In [ ]:
result_duration = {}

In [ ]:
for clf in cls:
    result_duration[clf]={}
    result_duration[clf][AllFeatures] = RunML_continue.nested_loocv(data,yList[:,2],classifier=clf,smote=True)
    result_duration[clf][SelectMicro] = RunML_continue.nested_loocv(X_FS,yList[:,2],classifier=clf,smote=True)
    result_duration[clf][Lasso_finetune] = RunML_continue.nested_loocv(selectedlasso_duration[0],yList[:,2],classifier=clf,smote=True)